# $\alpha$=0.1

In [2]:
import numpy as np
import torch
import torchvision.transforms as transforms              
from torchvision.datasets import CIFAR100      
from torch.utils.data import DataLoader
import timm
import detectors
from src.saps import split_data_set, saps_scores, saps_classification, eval_aps
from src.temperature_scaling import ModelWithTemperature

# check GPU status
print("Is CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load fine-tuned model
model = timm.create_model("resnet50_cifar100", pretrained=True)
model = model.to(device)

# reprocess the images from CIFAR
data_transform = transforms.Compose([
    transforms.ToTensor(),          # transfer to tensor
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))  # normalize
])
# load images from CIFAR10
dataset = CIFAR100(root="../../data", train=False, download=True, transform=data_transform)

temp_scal_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ModelWithTemperature(model, temperature=4.85).to(device)
model.set_temperature(temp_scal_loader)

model.eval()

# error rate
alpha = 0.1
lambda_ = 0.1
num_runs = 10

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("SAPS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # splite dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = saps_scores(model, calib_loader, alpha, lambda_, device)
    t_cal = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"t_cal = {t_cal}")

    # construct APS
    aps, aps_labels, true_labels = saps_classification(model, test_loader, t_cal, lambda_, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

Is CUDA available: True
Device count: 1
Device name: NVIDIA GeForce RTX 3060 Ti
Files already downloaded and verified
Before temperature - NLL: 1.073, ECE: 0.083
Optimal temperature: 4.692
After temperature - NLL: 2.953, ECE: 0.674
SAPS Classification, Start!

Running experiment 1/10...
t_cal = 0.28092794120311765
Average Prediction Set Size After APS in runs 1: 2.7568
Average Coverage Rate in runs 1: 0.9034

Running experiment 2/10...
t_cal = 0.2753291726112366
Average Prediction Set Size After APS in runs 2: 2.6974
Average Coverage Rate in runs 2: 0.905

Running experiment 3/10...
t_cal = 0.26039395928382886
Average Prediction Set Size After APS in runs 3: 2.557
Average Coverage Rate in runs 3: 0.896

Running experiment 4/10...
t_cal = 0.2714850842952734
Average Prediction Set Size After APS in runs 4: 2.6678
Average Coverage Rate in runs 4: 0.9018

Running experiment 5/10...
t_cal = 0.27600129544734997
Average Prediction Set Size After APS in runs 5: 2.7098
Average Coverage Rate in 

# $\alpha$=0.2

In [2]:
# error rate
alpha = 0.2
lambda_ = 0.1
num_runs = 10

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("SAPS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # splite dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = saps_scores(model, calib_loader, alpha, lambda_, device)
    t_cal = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"t_cal = {t_cal}")

    # construct APS
    aps, aps_labels, true_labels = saps_classification(model, test_loader, t_cal, lambda_, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

Files already downloaded and verified
Before temperature - NLL: 1.078, ECE: 0.084
Optimal temperature: 4.691
After temperature - NLL: 2.951, ECE: 0.676
SAPS Classification, Start!

Running experiment 1/10...
t_cal = 0.14076489210128784
Average Prediction Set Size After APS in runs 1: 1.4234
Average Coverage Rate in runs 1: 0.8014

Running experiment 2/10...
t_cal = 0.1396219044923783
Average Prediction Set Size After APS in runs 2: 1.4082
Average Coverage Rate in runs 2: 0.7992

Running experiment 3/10...
t_cal = 0.14207956194877625
Average Prediction Set Size After APS in runs 3: 1.4456
Average Coverage Rate in runs 3: 0.8052

Running experiment 4/10...
t_cal = 0.1429911375045777
Average Prediction Set Size After APS in runs 4: 1.46
Average Coverage Rate in runs 4: 0.811

Running experiment 5/10...
t_cal = 0.14386882781982438
Average Prediction Set Size After APS in runs 5: 1.474
Average Coverage Rate in runs 5: 0.8154

Running experiment 6/10...
t_cal = 0.1398466855287552
Average Pre

# $\alpha$=0.05

In [9]:
# error rate
alpha = 0.05
lambda_ = 0.03
num_runs = 10

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("SAPS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # splite dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = saps_scores(model, calib_loader, alpha, lambda_, device)
    t_cal = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"t_cal = {t_cal}")

    # construct APS
    aps, aps_labels, true_labels = saps_classification(model, test_loader, t_cal, lambda_, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

SAPS Classification, Start!

Running experiment 1/10...
t_cal = 0.25995366871356995
Average Prediction Set Size After APS in runs 1: 6.2928
Average Coverage Rate in runs 1: 0.9554

Running experiment 2/10...
t_cal = 0.242190832644701
Average Prediction Set Size After APS in runs 2: 5.692
Average Coverage Rate in runs 2: 0.9484

Running experiment 3/10...
t_cal = 0.24127786904573453
Average Prediction Set Size After APS in runs 3: 5.6844
Average Coverage Rate in runs 3: 0.947

Running experiment 4/10...
t_cal = 0.25152498483657837
Average Prediction Set Size After APS in runs 4: 6.0124
Average Coverage Rate in runs 4: 0.9498

Running experiment 5/10...
t_cal = 0.2474727354943753
Average Prediction Set Size After APS in runs 5: 5.8918
Average Coverage Rate in runs 5: 0.9498

Running experiment 6/10...
t_cal = 0.2327513560652734
Average Prediction Set Size After APS in runs 6: 5.4122
Average Coverage Rate in runs 6: 0.9424

Running experiment 7/10...
t_cal = 0.24783536270260817
Average Pr

# Result  

$\alpha$=0.1  
- Final Average **Prediction Set Size： 2.65**
- Final Average **Coverage: 90.02%**  

$\alpha$=0.2  
- Final Average **Prediction Set Size： 1.43**
- Final Average **Coverage: 80.25%**  

$\alpha$=0.05  
- Final Average **Prediction Set Size： 5.83**
- Final Average **Coverage: 94.86%**